### 사용자 기반 협업 필터링 이용한 제품 추천 시스템 / 물품명 추천 및 평가
#### 이름, 성별, 연령대, 계절 입력
작성한 모델에서 아무것도 안된다 -> 데이터 전처리 문제(타입, 호출 등)  
작성한 모델의 결과가 이상하다 -> 함수, 모델 학습 불량  

In [102]:
import numpy as np
import pandas as pd

# 협업필터링
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib

# 맑은 고딕 적용
matplotlib.rc("font", family = "Malgun Gothic")
# 음수 표시
matplotlib.rc('axes', unicode_minus=False)

#### 데이터 불러오기 및 처리

In [103]:
# 파일 경로 설정
# member_data_path = r"C:\Users\space\Desktop\교육\포스코 청년 Ai 및 빅데이터 아카데미\04. 본 교육\02. 빅데이터\02. 개인 및 팀 실습(프로젝트)\02. 팀\04. 빅데이터 프로젝트\01. 개인\06. 모델\01. 데이터\RFM_member.csv"
data_path = r"C:\Users\space\Desktop\교육\포스코 청년 Ai 및 빅데이터 아카데미\04. 본 교육\02. 빅데이터\02. 개인 및 팀 실습(프로젝트)\02. 팀\04. 빅데이터 프로젝트\01. 개인\06. 모델\01. 데이터\RFM_product.csv"

# 데이터 불러오기
# member_data = pd.read_csv(member_data_path, encoding='euc-kr')
data = pd.read_csv(data_path, encoding='euc-kr')

data.head()

,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,구매금액,구매매장,구매시각,...,주소대분류,주소소분류,계절,날짜,요일,연령대,구매시간대,R,F,M
0,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,22207.0,매장C,10:04,...,용인 수지구,풍덕천동,겨울,2023-01-06,금,40대,8~11시,11,75,1505970.0
1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,4403.0,매장C,10:09,...,용인 수지구,풍덕천동,겨울,2023-01-06,금,30대,8~11시,55,11,144730.0
2,743192512,정상회원,채소,양념채소,생강,생강(300g),1.0,7083.0,매장C,10:12,...,용인 수지구,성복동,겨울,2023-01-06,금,50대,8~11시,250,12,436583.0
3,176912064,정상회원,축산물,유제품,우유/산양유,유기농저지방우유(종이팩/900ml),1.0,7849.0,매장C,10:23,...,용인 수지구,풍덕천동,겨울,2023-01-06,금,60대,8~11시,2,92,1973350.0
4,87592064,정상회원,양념/가루,가루,양념가루,카레(매운맛: 100g),3.0,15507.0,매장C,10:29,...,용인 수지구,풍덕천동,겨울,2023-01-06,금,70대,8~11시,0,47,1420890.0


In [104]:
# 필요한 데이터만 추출
df1 = data[['성별', '연령대', '구매매장', '계절', '회원번호', '물품대분류', '물품소분류', '물품명', '구매수량', '구매금액']]
df2 = data[['성별', '연령대', '구매매장', '계절','회원번호']]
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288889 entries, 0 to 288888
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   성별      288889 non-null  object 
 1   연령대     288889 non-null  object 
 2   구매매장    288889 non-null  object 
 3   계절      288889 non-null  object 
 4   회원번호    288889 non-null  int64  
 5   물품대분류   288889 non-null  object 
 6   물품소분류   288889 non-null  object 
 7   물품명     288889 non-null  object 
 8   구매수량    288889 non-null  float64
 9   구매금액    288889 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 22.0+ MB


#### 협업필터링 모델을 이용한 유사성 계산 및 비교
입력받은 고객의 유사성과 기존 데이터 고객의 유사성을 각각 계산 후 비교하여 기존 데이터의 고객들을 추출

In [105]:
# '회원번호' 기준으로 중복된 행을 제거하고 첫 번째 데이터만 남김.
df_unique = df2.drop_duplicates(subset='회원번호', keep='first')

# Step 2: 범주형 데이터에 대한 One-hot Encoding
df_encoded = pd.get_dummies(df_unique[['성별', '연령대', '구매매장', '계절']], drop_first=False)
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11540 entries, 0 to 288785
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   성별      11540 non-null  object
 1   연령대     11540 non-null  object
 2   구매매장    11540 non-null  object
 3   계절      11540 non-null  object
 4   회원번호    11540 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 540.9+ KB


In [106]:
# 고객 특성 입력 받기 (새로운 고객 정보)
# 빈 DataFrame을 생성
sample_user_data = {'성별' : '', 
                    '연령대' : '', 
                    '구매매장' : '', 
                    '계절' : '', }  # 고객 정보 입력 받기

In [107]:
# 고객 정보 입력 받기
new_member = input("이름을 입력하세요 : ")

# 고객 정보 입력 받기
sample_user_data['성별'] = input("성별을 입력하세요 ('여' 또는 '남'): ")
print(f"성별: {sample_user_data['성별']}")  # 입력값 확인

sample_user_data['연령대'] = input("연령대를 입력하세요 (예: 20대, 30대): ")
print(f"연령대: {sample_user_data['연령대']}")  # 입력값 확인

sample_user_data['구매매장'] = input("구매매장을 입력하세요 (매장A, 매장B, 매장C, 매장D): ")
print(f"구매매장: {sample_user_data['구매매장']}")  # 입력값 확인

sample_user_data['계절'] = input("계절을 입력하세요 (여름, 겨울 등): ")
print(f"계절: {sample_user_data['계절']}")  # 입력값 확인

성별: 남
연령대: 50대
구매매장: 매장C
계절: 봄


In [108]:
# 데이터프레임으로 변환
sample_user_vector = pd.DataFrame([sample_user_data])
sample_user_vector = sample_user_vector.astype(str)

# 변환 확인
sample_user_vector.info()

# 입력받은 정보 인코딩 작업
sample_user_vector_encoded = pd.get_dummies(sample_user_vector, drop_first=False)

# 확인: 인코딩 후 데이터
print(sample_user_vector_encoded)

# 인코딩 결과 출력
sample_user_vector_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   성별      1 non-null      object
 1   연령대     1 non-null      object
 2   구매매장    1 non-null      object
 3   계절      1 non-null      object
dtypes: object(4)
memory usage: 164.0+ bytes
   성별_남  연령대_50대  구매매장_매장C  계절_봄
0  True     True      True  True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   성별_남      1 non-null      bool 
 1   연령대_50대   1 non-null      bool 
 2   구매매장_매장C  1 non-null      bool 
 3   계절_봄      1 non-null      bool 
dtypes: bool(4)
memory usage: 136.0 bytes


In [109]:
# 기존 고객 데이터와 새로운 고객의 특성 벡터 결합
# 새로운 고객 벡터와 기존 고객 데이터를 합침
all_data = pd.concat([df_encoded, sample_user_vector_encoded], ignore_index=True)

In [110]:
# 입력된 고객이 해당되지 않는 Column의 value Nan 수정
# NaN 값을 0으로 채우고, 모든 컬럼을 float 타입으로 변환
all_data = all_data.fillna(0).astype(float)

In [111]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11541 entries, 0 to 11540
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   성별_남       11541 non-null  float64
 1   성별_여       11541 non-null  float64
 2   연령대_10대    11541 non-null  float64
 3   연령대_20대    11541 non-null  float64
 4   연령대_30대    11541 non-null  float64
 5   연령대_40대    11541 non-null  float64
 6   연령대_50대    11541 non-null  float64
 7   연령대_60대    11541 non-null  float64
 8   연령대_70대    11541 non-null  float64
 9   연령대_80대    11541 non-null  float64
 10  연령대_90대이상  11541 non-null  float64
 11  구매매장_매장A   11541 non-null  float64
 12  구매매장_매장B   11541 non-null  float64
 13  구매매장_매장C   11541 non-null  float64
 14  구매매장_매장D   11541 non-null  float64
 15  계절_가을      11541 non-null  float64
 16  계절_겨울      11541 non-null  float64
 17  계절_봄       11541 non-null  float64
 18  계절_여름      11541 non-null  float64
dtypes: float64(19)
memory usage: 1.7 MB


In [112]:
# 코사인 유사도 계산
# 기존 고객과 새 고객 간 유사도 계산
cosine_sim = cosine_similarity(all_data[:-1], all_data[-1:])
similarity_scores = cosine_sim.flatten()

In [113]:
# 유사도 제대로 계산된지 확인
# 입력된 고객의 자기 자신과의 유사도 점수 출력 (항상 1)
input_customer_similarity = cosine_similarity(all_data[-1:], all_data[-1:])[0][0]
print(f"입력된 고객의 유사도 점수 (자기 자신과의 유사도): {input_customer_similarity}")

입력된 고객의 유사도 점수 (자기 자신과의 유사도): 1.0


In [114]:
# 입력한 고객과 유사한 기존 고객 찾기 (상위 N명의 유사 고객 추출)
top_n_similar_customers = similarity_scores.argsort()[-50:][::-1]

In [115]:
# 상위 N명의 유사도 점수와 회원번호 출력
print("\n입력된 고객과 유사한 상위 50명의 기존 고객:")
for idx in top_n_similar_customers:
    print(f"회원번호 {df_unique.iloc[idx]['회원번호']}: 유사도 점수 = {similarity_scores[idx]}")


입력된 고객과 유사한 상위 50명의 기존 고객:
회원번호 685786432: 유사도 점수 = 1.0
회원번호 878029056: 유사도 점수 = 1.0
회원번호 207964800: 유사도 점수 = 1.0
회원번호 78516224: 유사도 점수 = 1.0
회원번호 351110656: 유사도 점수 = 1.0
회원번호 747155072: 유사도 점수 = 1.0
회원번호 1579602624: 유사도 점수 = 1.0
회원번호 214052480: 유사도 점수 = 1.0
회원번호 1065314304: 유사도 점수 = 1.0
회원번호 619933696: 유사도 점수 = 1.0
회원번호 1475141376: 유사도 점수 = 1.0
회원번호 391224384: 유사도 점수 = 1.0
회원번호 506884736: 유사도 점수 = 1.0
회원번호 51207040: 유사도 점수 = 1.0
회원번호 1335735360: 유사도 점수 = 1.0
회원번호 272187968: 유사도 점수 = 1.0
회원번호 332179456: 유사도 점수 = 1.0
회원번호 481433408: 유사도 점수 = 1.0
회원번호 768118592: 유사도 점수 = 1.0
회원번호 763517568: 유사도 점수 = 1.0
회원번호 432659584: 유사도 점수 = 1.0
회원번호 1050286272: 유사도 점수 = 0.75
회원번호 743417088: 유사도 점수 = 0.75
회원번호 165696256: 유사도 점수 = 0.75
회원번호 207491520: 유사도 점수 = 0.75
회원번호 1009406016: 유사도 점수 = 0.75
회원번호 349896832: 유사도 점수 = 0.75
회원번호 775887808: 유사도 점수 = 0.75
회원번호 393538816: 유사도 점수 = 0.75
회원번호 166316160: 유사도 점수 = 0.75
회원번호 219267840: 유사도 점수 = 0.75
회원번호 99257024: 유사도 점수 = 0.75
회원번호 775564864: 유사도 점수 = 0.75


In [116]:
# 유사도가 낮은 기존 고객 찾기 (하위 N명의 유사 고객 추출)
bottom_n_similar_customers = similarity_scores.argsort()[:10]  # 하위 10명의 유사한 고객

# 하위 10명의 유사도 점수와 회원번호 출력
print("\n입력된 고객과 유사도가 가장 낮은 하위 10명의 기존 고객:")
for idx in bottom_n_similar_customers:
    print(f"회원번호 {df_unique.iloc[idx]['회원번호']}: 유사도 점수 = {similarity_scores[idx]}")



입력된 고객과 유사도가 가장 낮은 하위 10명의 기존 고객:
회원번호 567412608: 유사도 점수 = 0.0
회원번호 856341696: 유사도 점수 = 0.0
회원번호 861169152: 유사도 점수 = 0.0
회원번호 1187307328: 유사도 점수 = 0.0
회원번호 1038391168: 유사도 점수 = 0.0
회원번호 263908352: 유사도 점수 = 0.0
회원번호 970164608: 유사도 점수 = 0.0
회원번호 1143268160: 유사도 점수 = 0.0
회원번호 445772224: 유사도 점수 = 0.0
회원번호 1241886720: 유사도 점수 = 0.0


In [117]:
# Step 8: 유사한 고객들의 회원번호 출력
similar_customer_ids = df_unique.iloc[top_n_similar_customers]['회원번호'].values
print(f"\n유사한 고객들의 회원번호 (상위 50명): {similar_customer_ids}")


유사한 고객들의 회원번호 (상위 50명): [ 685786432  878029056  207964800   78516224  351110656  747155072
 1579602624  214052480 1065314304  619933696 1475141376  391224384
  506884736   51207040 1335735360  272187968  332179456  481433408
  768118592  763517568  432659584 1050286272  743417088  165696256
  207491520 1009406016  349896832  775887808  393538816  166316160
  219267840   99257024  775564864   72454528  348764672  463188928
 1478608384 1500629824 1188716032  850339392  351079104  127277056
  505635648  766088128 1582765248 1560541504   58484416 1005729280
 1459120384 1160584640]


In [118]:
print(similar_customer_ids)

[ 685786432  878029056  207964800   78516224  351110656  747155072
 1579602624  214052480 1065314304  619933696 1475141376  391224384
  506884736   51207040 1335735360  272187968  332179456  481433408
  768118592  763517568  432659584 1050286272  743417088  165696256
  207491520 1009406016  349896832  775887808  393538816  166316160
  219267840   99257024  775564864   72454528  348764672  463188928
 1478608384 1500629824 1188716032  850339392  351079104  127277056
  505635648  766088128 1582765248 1560541504   58484416 1005729280
 1459120384 1160584640]


#### 도출된 유사도가 높은 고객들의 품목별 구매수량 및 매출을 이용한 추천 제품 도출  
- 상위 유사도 고객들 50명 대상으로 도출.  
- 품목별 구매수량 및 매출 순위를 50위까지 도출.  
- 1 ~ 50위까지 순위 그대로 점수 부여. / 1위는 1점, 50위는 50점.  
- 이후 가중치를 이용해 점수화.  
- Y = 0.4 x 구매수량 + 0.6 x 매출 -> 구매수량보다는 품목별 매출이 더 중요하다 판단하여 가중치를 좀 더 부여  
- 도출된 점수가 낮을 수록 상위 추천 제품이며 5개의 물품명을 추천!

In [119]:
# 유사한 고객의 구매 데이터를 필터링
similar_customers_data = df1[df1['회원번호'].isin(similar_customer_ids[:100])]

In [120]:
# 입력된 고객의 성별, 계절, 연령대, 구매매장에 속한 품목들만 유사 고객들의 구매 데이터로 필터링
filtered_similar_customers_data = similar_customers_data[
    (similar_customers_data['성별'] == sample_user_data['성별']) &
    (similar_customers_data['계절'] == sample_user_data['계절']) &
    (similar_customers_data['연령대'] == sample_user_data['연령대']) &
    (similar_customers_data['구매매장'] == sample_user_data['구매매장'])
]

# 필터링된 데이터에서 품목별 총 구매수량과 총 매출 계산
item_summary = filtered_similar_customers_data.groupby('물품명').agg(
    총구매수량=('구매수량', 'sum'),
    총매출=('구매금액', 'sum')
).reset_index()

In [121]:
# 구매수량과 매출 각각 순위 매기기
item_summary['구매수량순위'] = item_summary['총구매수량'].rank(method='dense', ascending=False)
item_summary['매출순위'] = item_summary['총매출'].rank(method='dense', ascending=False)

In [122]:
# 점수 부여 (순위가 높을수록 낮은 점수, 1등은 1점부터 50위는 50점까지)
item_summary['구매수량점수'] = item_summary['구매수량순위'].apply(lambda x: x if x <= 50 else 50)
item_summary['매출점수'] = item_summary['매출순위'].apply(lambda x: x if x <= 50 else 50)


In [123]:
# 가중치 적용 및 최종 점수 계산 (점수가 낮을수록 추천 순위가 높아짐)
item_summary['최종점수'] = (item_summary['구매수량점수'] * 0.4) + (item_summary['매출점수'] * 0.6)

In [124]:
# 최종 점수 기준으로 상위 5개 품목 추천
recommended_items = item_summary.sort_values('최종점수').head(1)  # 최종 점수 낮은 순으로 정렬 후 상위 5개 추출

# 추천 품목 상위 3개 출력
print("추천 품목 상위 1개:")
print(recommended_items[['물품명', '최종점수']])

추천 품목 상위 1개:
                물품명  최종점수
36  얼음식혜(1000ml/서정)   1.0


In [125]:
# 추천 제품 출력
print(f"{new_member} 고객님께 다음과 같은 제품를 추천드립니다.")
print(", ".join(recommended_items['물품명']))

홍길동 고객님께 다음과 같은 제품를 추천드립니다.
얼음식혜(1000ml/서정)


#### 모델 성능 평가
- 유사도 기반 평가지표 이용  
- 유사도 점수를 사용하여 추천 품질을 간접적으로 평가!

In [126]:
# Step 1: 상위 100명의 유사 고객의 유사도 점수를 추출
top_100_similarities = similarity_scores[top_n_similar_customers]

# Step 2: 유사도 기반 평가지표 계산
'''# 1) 상위 100명의 유사 고객 평균 유사도
average_similarity = np.mean(top_100_similarities)
print(f"상위 50명 유사 고객 평균 유사도: {average_similarity:.2f}")

# 2) 상위 100명 유사도 점수의 분포 및 비율 계산
# 상위 유사도 비율 (예: 유사도가 0.8 이상인 고객 비율)
high_similarity_ratio = np.sum(top_100_similarities >= 0.8) / len(top_100_similarities)
print(f"유사도 0.8 이상인 고객 비율: {high_similarity_ratio:.2f}")'''

# 3) 상위 5개 추천 품목과 평균 유사도 매칭
# 상위 5개 추천 품목의 고객의 평균 유사도와 비교
recommended_items_customers = df1[df1['물품명'].isin(recommended_items['물품명'])]['회원번호']

# 중복된 회원번호 제거하여 고유 고객 리스트 생성
unique_recommended_items_customers = recommended_items_customers.drop_duplicates()

# 고유한 추천 제품 구매 고객들의 유사도 추출
recommended_customers_similarity = similarity_scores[df_unique['회원번호'].isin(unique_recommended_items_customers)]
recommended_customers_avg_similarity = np.mean(recommended_customers_similarity)

# Step 3: 결과 해석
print("\n유사도 기반 평가지표:")
# print(f"- 상위 50명 유사 고객 평균 유사도: {average_similarity:.2f}")
# print(f"- 유사도 0.8 이상인 고객 비율: {high_similarity_ratio:.2f}")
print(f"- 추천 제품을 실제 구매한 고객들과 입력된 고객 사이의 평균 유사도: {recommended_customers_avg_similarity:.2f}")


유사도 기반 평가지표:
- 추천 제품을 실제 구매한 고객들과 입력된 고객 사이의 평균 유사도: 0.24


즉, 추천 제품을 실제 구매한 고객들과 입력된 고객 사이의 평균 유사도  
여성, 50대, 매장C, 봄 입력시의 평균 유사도 = 0.57 정도!  
만약 데이터가 적은 남자, 50대로 기재한다면(매장C, 봄)은 평균 유사도가 0.24   
-> 주요 고객층인 여성에 대해서는 추천이 어느정도 되고 있다는 것! 